# 加载图像

In [1]:
import mxnet as mx
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

img = cv2.imread('../images/vga.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def swap_channels(img):
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)0
    # swap channels [height, width, channel] to [channel, height, width]
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

img = swap_channels(img)
print('image size:', img.shape)

OSError: libcudnn.so.5: cannot open shared object file: No such file or directory

# 读取模型文件

In [ ]:
sym, arg_params, aux_params = mx.model.load_checkpoint('../models/mtcnn/pnet', 0)

为了清晰的显示模型所有层输出, 我们将其打印出来.

In [ ]:
all_layers = sym.get_internals()
# for layer in all_layers.list_arguments():
#     if layer != "data":
#         print("{}\t{}".format(layer, arg_params[layer].shape))
for key in sorted(arg_params.keys()):
    print('{}\t{}'.format(key, arg_params[key].shape))

# 获取 feature map
从 all_layers.list_outputs() 打印结果可以看出, 凡带有 **output** 项的就是特征图.

## 输出各层 feature map 的维度 

In [ ]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

print("Size of feature map:\n")
layer_info = all_layers.list_outputs()
for name in layer_info:
    if name.find('output') != -1:  # 查找包含 "output" 的字符串
        layer_sym = all_layers[name]
        mod = mx.mod.Module(symbol=layer_sym, label_names=None, context=mx.cpu())
        mod.bind(for_training=False, data_shapes=[('data', img.shape)])
        mod.set_params(arg_params=arg_params, aux_params=aux_params)
        mod.forward(Batch([mx.nd.array(img)]))
        out_map = mod.get_outputs()[0].asnumpy()
        print('{}\t{}'.format(name, out_map.shape))

<font color=orchid>**Another way to get the dimentions of parameters and feature maps**:</font>  

In [60]:
data_shape_dict = dict([('data', img.shape)])
arg_shape, out_shape, aux_shape = sym.infer_shape(**data_shape_dict)
arg_shape_dict = dict(zip(sym.list_arguments(), arg_shape))
aux_shape_dict = dict(zip(sym.list_auxiliary_states(), aux_shape))

# print the shape of parameters
print("Shape of parameters:")
for key in arg_shape_dict.keys():
    print('{}\t{}'.format(key, arg_shape_dict[key]))

# infer the shape of feature maps
print("\nShape of feature maps:")
for name in all_layers.list_outputs():
    if name.find('output') != -1:  # 查找包含 "output" 的字符串
        arg_shape, out_shape, aux_shape = all_layers[name].infer_shape(**data_shape_dict)
        layer_shape_dict = dict(zip(all_layers[name].list_outputs(), out_shape))
        print(layer_shape_dict)

Shape of parameters:
data	(1, 3, 480, 640)
conv1_weight	(10, 3, 3, 3)
conv1_bias	(10,)
prelu1_gamma	(10,)
conv2_weight	(16, 10, 3, 3)
conv2_bias	(16,)
prelu2_gamma	(16,)
conv3_weight	(32, 16, 3, 3)
conv3_bias	(32,)
prelu3_gamma	(32,)
cls_conv_weight	(2, 32, 1, 1)
cls_conv_bias	(2,)
bbox_reg_weight	(4, 32, 1, 1)
bbox_reg_bias	(4,)

Shape of feature maps:
{'conv1_output': (1, 10, 478, 638)}
{'prelu1_output': (1, 10, 478, 638)}
{'pool1_output': (1, 10, 239, 319)}
{'conv2_output': (1, 16, 237, 317)}
{'prelu2_output': (1, 16, 237, 317)}
{'conv3_output': (1, 32, 235, 315)}
{'prelu3_output': (1, 32, 235, 315)}
{'cls_conv_output': (1, 2, 235, 315)}
{'cls_output': (1, 2, 235, 315)}
{'bbox_reg_output': (1, 4, 235, 315)}


## 显示 feature map 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

# get certain layers output
layer_names = ['conv2_output', 'prelu2_output', 'conv3_output', 'cls_output', 'bbox_reg_output']
num_items = len(layer_names)

fig = plt.figure()
for i in range(num_items):
    conv_sym = all_layers[layer_names[i]]
    mod = mx.mod.Module(symbol=conv_sym, label_names=None, context=mx.cpu())
    mod.bind(for_training=False, data_shapes=[('data', img.shape)])
    mod.set_params(arg_params=arg_params, aux_params=aux_params, allow_extra=True)
    mod.forward(Batch([mx.nd.array(img)]))
    out_map = mod.get_outputs()[0].asnumpy()
    
    # show
    ax = fig.add_subplot(1, num_items, i+1)  
    ax.imshow(out_map[0,0, :, :], cmap='gray')   
    ax.set_title(layer_names[i]) 
    plt.axis("off")
    
plt.show()  